In [ ]:
# current submission is .79
# Things I'd like to do:
    # try unsupervised ML?
    # Try a NN or reinforcement or something?
    # Learn how to plot data and make charts?
    # Can we use the cabin column in a meaningful way for rows that have that?
    # find other ways to handle NaN Age?
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.preprocessing import OneHotEncoder, minmax_scale
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
#!kaggle competitions download -c titanic
#!kaggle datasets download -d zynicide/wine-reviews # Use this to get data if you want it.

In [ ]:
og_df = pd.read_csv('./data/train.csv', engine='python')
og_df

In [ ]:
# just rounds the models prediction to 0 or 1.
def get_model_val(df_row, model):
    if(model.predict([df_row])[0] > .5):
        return 1
    else:
        return 0

def clean_data(df):
    # Transformations here.
    new_df = df.copy() 
    new_df = new_df.drop(columns=['Name', 'Ticket', 'Cabin'])

    new_df['Age'] = new_df['Age'].fillna(new_df.Age.median())

    new_df = pd.get_dummies(new_df, columns=['Sex', 'Embarked', 'Pclass'])
    new_df = pd.DataFrame(minmax_scale(new_df), columns=new_df.columns)

    return new_df

def test_data(test_df, ans_df):
    # For local testing so I don't have to submit all the time.
    correct = 0
    total = 0 
    for index, row in test_df.iterrows():
        if row['PassengerId'] in ans_df['PassengerId'].values:
            if ans_df.loc()[ans_df['PassengerId'] == row['PassengerId']].iloc[0]['Survived'] == row['Survived']:
                correct = correct +1
        total += 1 
    return "{} out of {} for a % of {}".format(correct, total, correct/total)

In [ ]:
df = clean_data(og_df)
df_passenger_id = og_df['PassengerId'].copy()
df_labels = df['Survived'].copy()
corr_matrix = df.corr()
df = df.drop(columns=['Survived', 'PassengerId'])

In [ ]:
corr_matrix['Survived'].sort_values(ascending=False)

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(df,df_labels)
model = lin_reg
lin_reg.predict([df.iloc[1]])[0] # sanity check

In [ ]:
log_reg = LogisticRegression(C=.01)
log_reg.fit(df,df_labels)
model = log_reg
log_reg.predict([df.iloc[1]])[0] # sanity check

In [ ]:
rid_reg = Ridge()
rid_reg.fit(df,df_labels)
model = rid_reg
rid_reg.predict([df.iloc[1]])[0] # sanity check

In [ ]:
las_reg = Lasso(alpha=.001, max_iter=100)
las_reg.fit(df,df_labels)
model = las_reg
las_reg.predict([df.iloc[1]])[0] # sanity check

In [ ]:
tree_reg = DecisionTreeRegressor(max_depth=6)
tree_reg.fit(df, df_labels)
model = tree_reg
tree_reg.predict([df.iloc[1]])[0] # sanity check

In [ ]:
forest_reg = RandomForestRegressor()
#forest_reg.fit(df, df_labels)
#model = forest_reg
#forest_reg.predict([df.iloc[1]])[0] # sanity check

# hyper parameter optimization
param_grid = {'bootstrap': [True, False],
 'max_depth': [4, 6, 8, 10, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2],
 'min_samples_split': [2, 5],
 'n_estimators': [200, 400, 600, 800, 1000]}

r_search = RandomizedSearchCV(forest_reg, cv=5, param_distributions=param_grid, scoring='neg_mean_squared_error',
                            n_iter=25, verbose=2, n_jobs=-1)
r_search.fit(df, df_labels)
r_search.predict([df.iloc[1]])[0]

In [ ]:
model = r_search
r_search.best_params_

In [ ]:
knn = KNeighborsClassifier(n_neighbors=2) # can play with this a bit.
knn.fit(df,df_labels)
model = knn
knn.predict([df.iloc[1]])[0] # sanity check

In [ ]:
tr_og_df = pd.read_csv('./data/test.csv', engine='python')
tr_df_passengerId = tr_og_df['PassengerId'].copy()
tr_df = clean_data(tr_og_df)
tr_df['Fare'] = tr_df['Fare'].fillna(tr_df.Fare.median())
tr_df = tr_df.drop(columns=['PassengerId'])
tr_df['Survived'] = tr_df.apply(lambda row: get_model_val(row, model), axis=1)
final_df = pd.DataFrame(list(zip(tr_df_passengerId, tr_df['Survived'])),columns=['PassengerId', 'Survived'])

In [ ]:
final_df.to_csv('output_hpo_rf.csv', index=False)
#!kaggle competitions submit -c titanic -f ./output.csv

In [ ]:
ans_df = pd.read_csv('./data/answers.csv', engine='python')
ans_df = ans_df.rename({1: 'PassengerId', 2:'Survived'}, axis='columns')

test_df = tr_og_df.merge(ans_df, left_on="Name", right_on="name")
test_df_only_survived = test_df[['survived', 'PassengerId']].copy()
test_df_only = test_df_only_survived.rename({'survived': 'Survived'}, axis='columns')
ans_df = test_df_only
ans_df = ans_df.drop(1) # Something weird happens and we have to drop this one.
print(test_data(final_df, ans_df))
#ans_df = ans_df[ans_df['name'] == tr_og_df['Name']]

#ans_df.drop(ans_df.index[0:892], inplace=True) # We can't do this as test is not sorted.
#final_df = final_df.drop([417], axis="index") # We have to do this before testing for some reason.

In [ ]:
best_params = {'n_estimators': 1000,
 'min_samples_split': 2,
 'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'max_depth': 90,
 'bootstrap': False}
score = 